In [1]:
# This is not a production ready code. It's a working POC.
# Needs error validations and data tests added

from google.oauth2 import service_account
from googleapiclient.discovery import build
from google.cloud import bigquery
import os
import json
import pandas as pd
import pandas_gbq

In [2]:
# path to your json for GA keys
CLIENT_SECRETS_PATH ="XXXX"
VIEW_ID = "196762668"
# path to BQ json keys
# BQ user should have 'BigQuery Job User' permissoins added
CREDENTIALS_PATH = 'XXXXXXX'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = CREDENTIALS_PATH


In [10]:
def initialize_analyticsreporting():

  credentials = service_account.Credentials.from_service_account_file(CLIENT_SECRETS_PATH,
                                scopes = ['https://www.googleapis.com/auth/analytics.readonly'])
  # Build the service object.
  analytics = build('analyticsreporting', 'v4', credentials=credentials)

  return analytics

def get_report(analytics):
  # This is a sample report. Should be factored out eventually
  return analytics.reports().batchGet(
      body={
        'reportRequests': [
        {
          'viewId': VIEW_ID,
          'dateRanges': [{'startDate': '2019-06-10', 'endDate': '2019-12-31'}],
          'metrics': [{'expression': 'ga:users'},
                      {'expression':'ga:sessions'}],
          'dimensions': [{'name': 'ga:date'}, {'name': 'ga:browser'}]
        }]
      }
  ).execute()


def get_response(response):
  # Parses and returns the Analytics Reporting API V4 response as an array of dictionaries representing 
  # each row to be added to a BQ table
  bq_data = []
  for report in response.get('reports', []):
    columnHeader = report.get('columnHeader', {})
    dimensionHeaders = columnHeader.get('dimensions', [])
    metricHeaders = columnHeader.get('metricHeader', {}).get('metricHeaderEntries', [])
    rows = report.get('data', {}).get('rows', [])
    for row in rows:
      bq_row = {}
      dimensions = row.get('dimensions', [])
      dateRangeValues = row.get('metrics', [])
      # print (dimensions)
      # print (dateRangeValues)
      for header, dimension in zip(dimensionHeaders, dimensions):
        #print (header + ': ' + dimension)
        bq_row[header] = dimension

      for values in dateRangeValues:
        for metricHeader, value in zip(metricHeaders, values.get('values')):
          bq_row[metricHeader.get('name')] = value
      bq_data.append(bq_row)

  return bq_data

In [12]:
analytics = initialize_analyticsreporting()
response = get_report(analytics)
bq_data = get_response(response)
# convert a list of dictionaries to json
bq_data = json.dumps(bq_data)
# bq only weirdly accepts new line separated JSON so to avoid saving the file
# and converting it, I use pandas
df = pd.read_json(bq_data)
pandas_gbq.to_gbq(df, 'ga_to_bq.u_b_c', project_id='data-engineering-390014',
                  if_exists = 'append')